<a href="https://colab.research.google.com/github/diptoromeo/LangChain-based-LLM-for-ChatGPT/blob/main/06_LangChain_VectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain
!pip install langchain-community langchain-core
!pip install tiktoken
!pip install sentence_transformers
!pip install pypdf
!pip install chromadb #vectore store DB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nv

In [11]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def token_len(text):
    encoding = tiktoken.get_encoding('cl100k_base')
    tokens = encoding.encode(text)
    return len(tokens)

In [12]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf")
pages = loader.load_and_split()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=token_len
    )

docs = text_splitter.split_documents(pages)

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
db = Chroma.from_documents(docs, hf)

In [17]:
query = '장기저탄소발전전략'
docs = db.similarity_search(query)

print(docs[0].page_content)
print(len(docs))

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최
4


In [18]:
db2 = Chroma.from_documents(docs, hf, persist_directory='./chroma_db') #DB와 파일 같이 저장

In [19]:
db3 = Chroma(persist_directory='./chroma_db', embedding_function=hf) #DB에서 저장된 파일이 풀어온다.
docs = db3.similarity_search(query)

print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [28]:
docs = db3.similarity_search_with_relevance_scores(query, k=3)

print('가장 유사한 무서: \n\n {}'.format(docs[0][0].page_content))
print('\n유사도:{}'.format(docs[0][1]))

가장 유사한 무서: 

 05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최

유사도:0.5288704602664074
